In [ ]:
import numpy as np
from numpy import array as arr

import pandas
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib as mpl
# import pylandau
from scipy.optimize import curve_fit
import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import uarray as uarr
from uncertainties.unumpy import nominal_values as val
from uncertainties.unumpy import std_devs as dev
from uncertainties import ufloat as uf

# import my plotting stuff
import sys
sys.path.append('./PythonHelpers/')
from PlotLib import Plotting
import PlotLib.Histogramming as Hist
import PlotLib.Plotting as Plot
from CSVimporter.importer import load_run
from CSVimporter.importer import dict_to_arr

def fitfunc_gauss(x, A, mu, sigma):
    return A * np.exp(-0.5*((x-mu)/sigma)**2)

In [ ]:
loadEvents = 100000
runIDs = arr([637,635,639,633,638,636,640,634])
# runIDs = arr([634])

runs = {}
for runID in runIDs:
    runs[runID] = load_run(runID, loadEvents*4)
    runs[runID]["nEvts"] = min(runs[runID]["nEvts"],loadEvents)
    
# runs[runID]["data"][evt,pixel,M[key]]
M = runs[runIDs[0]]["M"]
scalefactor = arr([1,100000/58975,1,1,1,1,1,1])

In [ ]:
binRange = [0.025,0.2]
binN = 40
key = "AmplitudeEst"
runIDs = arr([637,635,639,633,638,636,640,634])
use_risetime_regions = True


binWidth = (binRange[1]-binRange[0])/binN
fitRangeBinsLow = arr([
    [27,25,24,28],
    [26,25,23,26],
    [25,23,22,24],
    [24,21,20,23],
    [22,21,17,23],
    [22,20,16,22],
    [21,19,16,21],
    [21,18,15,20],
])
fitRangeBinsHigh = arr([ [35]*4, [34]*4, [34]*4, [34]*4, [34]*4, [34]*4, [34]*4, [38]*4 ])

risetime_regions = {
    637: [[6.,8],[6.,8],[6.,8],[6.,8]],
    635: [[6.,8.5],[6.,8.5],[6.,8.5],[6.,8.5]],
    639: [[5.5,7.5],[6.,8.5],[6.,8],[6.,8.5]],
    633: [[5.5,7.5],[6.,8],[5.5,7.5],[6.,8]],
    638: [[5.5,7.5],[5.5,8],[5.5,7.5],[5.5,8]],
    636: [[5.5,7],[5.5,8],[5.5,7.5],[5.5,8]],
    640: [[5.5,7],[5.5,8],[5.5,7.5],[5.5,8]],
    634: [[5.5,7],[5.5,7.7],[5.5,7.3],[5.5,7.7]],
}

fig, axs = Hist.create_fig(2,4, [10,12])
pixMap = [1,3,0,2]
PixNames = arr(["Pix 0,0", "Pix 0,1", "Pix 1,0", "Pix 1,1"])
    
for i,runID in enumerate(runIDs):
    runs[runID]["charge_cal"] = uarr([0]*4, [0]*4)
    for i_pix in range(4):
        if use_risetime_regions:
            risetime_region = risetime_regions[runID][i_pix]
            mask1 = runs[runID]["data"][:,i_pix,M["Risetime"]] > risetime_region[0]
            mask2 = runs[runID]["data"][:,i_pix,M["Risetime"]] < risetime_region[1]
            mask = np.logical_and(mask1, mask2)

        entries = runs[runID]["data"][:,i_pix,M[key]][mask]
        hist, bins = np.histogram(entries, bins=binN, range=binRange)
        
        if False:
            fitMask = np.zeros(len(bins)-1, dtype=bool)
            for j in range(len(bins)-1):
                if j > fitRangeBinsLow[i,i_pix] and j < fitRangeBinsHigh[i,i_pix]:
                    fitMask[j] = True
            p0 = arr([1000, 0.13, 0.01])
            popt, perr, chi2, ndeg = Hist.fit_wrapper(hist, bins, fitMask, fitfunc_gauss, p0)
            runs[runID]["charge_cal"][i_pix] = 1620 / uf(popt[1], perr[1])
            # print(" rms",popt[2])
            
            fitX = np.arange(bins[:-1][fitMask][0]+binWidth/2, bins[:-1][fitMask][-1]+3/2*binWidth, binWidth/20)
            x = np.arange(binRange[0], binRange[1],(binRange[1]-binRange[0])/1000)     
        
            axs[i].plot(fitX, fitfunc_gauss(fitX,*popt), color="black", lw=1, label="")
            axs[i].plot(x, fitfunc_gauss(x,*popt), color="black", lw=.5, ls=":", label="")
        
            axs[i].axvline(popt[1]+popt[2], color="C"+str(i_pix), lw=1, ls="--", alpha=0.6)
            axs[i].axvline(popt[1]-popt[2], color="C"+str(i_pix), lw=1, ls="--", alpha=0.6)
            
            label = PixNames[i_pix]+" N="+str(int(np.sum(hist)/1000))+"k" +"\n  "+r"$\mu$"+"={:.1u}".format(uf(popt[1], perr[1])) +"\n  "+r"chi2red"+"={:.1f} / {:n}".format(chi2,ndeg)
        else:
            popt = [0,0,0]
            label = PixNames[i_pix]+" N="+str(int(np.sum(hist)/1000))+"k"
        Hist.draw(axs[i], hist, bins, color="C"+str(i_pix), label=label, fill_alpha=0.1)
    axs[i].set_xlim(binRange)

subtitles = dict_to_arr(runs, runIDs, "krum_bias_trim")
subtitles = ["krum_bias_trimming = "+str(subtitles[i])+"nA" for i in range(len(subtitles))]
Hist.finalize(runs[runID], fig, axs, "Amplitude [V]", "Entries", "Amplitude", subtitles, measurement="Fe55_all", logy=False)
fig.subplots_adjust(hspace=.15, wspace=.09, top=0.935)
fig.savefig("/home/jona/DESY/analysis_python/output/ChargeCalibration/Amplitude.pdf")
fig.savefig("/home/jona/DESY/analysis_python/output/ChargeCalibration/png/Amplitude.png", dpi=400)

In [ ]:
def get_mask_noChargeSharing(data, threshold):
    # data[evt,pixel,M[key]]
    mask = np.ones(len(data), dtype=bool)
    key = "Amplitude"
    for i_evt in range(len(data)):
        if np.sum(data[i_evt,:,M[key]] > threshold) > 1:
            mask[i_evt] = False
    return mask

fig, axs = Hist.create_fig(2,2,[8,6])
# runIDs = arr([637,635,639,633,638,636,640,634])
runIDs = arr([637,639,636,634])
binRange = [25,190]
binN = 40
krum_bias_trims = dict_to_arr(runs, runIDs, "krum_bias_trim")
colors = Hist.get_color_range(krum_bias_trims, maxLightness=0.8)
# colors = Hist.get_color_range(len(runIDs), maxLightness=0.8)

for i_runID, runID in enumerate(runIDs):
    mask_noChargeSharing = get_mask_noChargeSharing(runs[runID]["data"], 0.025)
    np.invert(mask_noChargeSharing, out=mask_noChargeSharing)
    
    for i_pix in range(4):
        entries = runs[runID]["data"][:,pixMap[i_pix],M[key]]
        # entries = entries[mask_noChargeSharing]
        
        
        hist, bins = np.histogram(entries*1000, bins=binN, range=binRange)
        
        label = "i="+str(runs[runID]["krum_bias_trim"])+"nA" + " (N="+str(int(np.sum(hist)/1000))+"k)"
        Hist.draw(axs[i_pix], hist, bins, color=colors[i_runID], label=label, fill_alpha=0.1)
        Fe55peak = 1 / (runs[runID]["charge_cal"][i_pix].nominal_value / 1620) * 1000
        axs[i_pix].axvline(Fe55peak, color=colors[i_runID], ls="--", alpha=0.7)

axs[0].set_xlim(binRange)
for i_pix in range(4):
    # axs[i_pix].set_xlim(binRange)
    title = PixNames[i_pix]
    axs[i_pix].set_title(PixNames[i_pix])

subtitles = ""
Hist.finalize(runs[runID], fig, axs, "Amplitude [mV]", "Entries", "Amplitude (krum_bias_trimming scan)", subtitles, measurement="Fe55_all", logy=False)
fig.savefig("output/ChargeCalibration/Amplitude_pixWise.pdf")
fig.savefig("output/ChargeCalibration/png/Amplitude_pixWise.png", dpi=400)

In [ ]:
fig, ax = Hist.create_fig(1,1,[6,4])

runIDs = arr([637,635,639,633,638,636,640,634])
for i_pix in range(4):
    krum_bias_trimS = dict_to_arr(runs, runIDs, "krum_bias_trim")
    charge_calS = uarr([val(runs[runID]["charge_cal"][i_pix]) for runID in runIDs],[dev(runs[runID]["charge_cal"][i_pix]) for runID in runIDs])
    # Hist.draw(axS, val(charge_cal), np.arange(len(runIDs)), color="C"+str(i_pix), label=PixNames[i_pix], fill_alpha=0.1)
    ax.errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color="C"+str(i_pix), label=PixNames[i_pix], capsize=3, markersize=3)

ax.set_xticks(krum_bias_trimS)
Hist.finalize(runs[runID], fig, ax, "krum_bias_trim [nA]", r"Calibration [$e^{-}/$ V]", "Charge Calibration Factors", subtitles, measurement="Fe55_all", logy=False)
fig.savefig("output/ChargeCalibration/CalibrationFactors.pdf")
fig.savefig("output/ChargeCalibration/png/CalibrationFactors.png", dpi=400)

In [ ]:
fig, ax = Hist.create_fig(1,1,[6,4])

runIDs = arr([637,635,639,633,638,636,640,634])
for i_pix in range(4):
    krum_bias_trimS = dict_to_arr(runs, runIDs, "krum_bias_trim")
    charge_calS = uarr([val(runs[runID]["charge_cal"][i_pix]) for runID in runIDs],[dev(runs[runID]["charge_cal"][i_pix]) for runID in runIDs])
    charge_calS = 1 / (charge_calS / 1620) * 1000
    ax.errorbar(krum_bias_trimS, val(charge_calS), yerr=dev(charge_calS), fmt="o-", color="C"+str(i_pix), label=PixNames[i_pix], capsize=3, markersize=3)

ax.set_xticks(krum_bias_trimS)
Hist.finalize(runs[runID], fig, ax, "krum_bias_trim [nA]", "Fe55 Peak Position [mV]", "Fe55 Peak Position", subtitles, measurement="Fe55_all", logy=False)
fig.savefig("output/ChargeCalibration/Fe55PeakPosition.pdf")
fig.savefig("output/ChargeCalibration/png/Fe55PeakPosition.png", dpi=400)

In [ ]:
def get_calibration_factor_(runs_, krum_bias_trim, pix):
    runIDs = arr([637,635,639,633,638,636,640,634])
    krum_bias_trims = dict_to_arr(runs_, runIDs, "krum_bias_trim")
    runID = runIDs[np.where(krum_bias_trims == krum_bias_trim)[0][0]]
    return runs_[runID]["charge_cal"][pix]

def get_calibration_factorS(runs_, krum_bias_trimS):
    if len(krum_bias_trimS) != 4:
        print("Error: krum_bias_trimS must have length 4, coresponding to the 4 pixels")
    return arr([get_calibration_factor_(runs_, krum_bias_trimS[pix], pix).nominal_value for pix in range(4)])

print("184: ", get_calibration_factorS(runs, arr([4.57,6.78,5.53,5.53])))
print("190: ", get_calibration_factorS(runs, arr([2.32,3.30,3.30,3.30])))
print("192: ", get_calibration_factorS(runs, arr([6.78,9.91,9.91,9.91])))



In [ ]:
fig, axs = Hist.create_fig(2,2,[8,6])
# runIDs = arr([637,635,639,633,638,636,640,634])
runIDs = arr([637,639,636,634])
key = "ToT"
binRange = [0,850]
binN = 60
krum_bias_trims = dict_to_arr(runs, runIDs, "krum_bias_trim")
colors = Hist.get_color_range(krum_bias_trims, maxLightness=0.8)
# colors = Hist.get_color_range(len(runIDs), maxLightness=0.8)



for i_runID, runID in enumerate(runIDs):
    for i_pix in range(4):
        entries = runs[runID]["data"][:,pixMap[i_pix],M[key]]
        hist, bins = np.histogram(entries, bins=binN)
        
        label = "i="+str(runs[runID]["krum_bias_trim"])+"nA" + " (N="+str(int(np.sum(hist)/1000))+"k)"
        Hist.draw(axs[i_pix], hist, bins, color=colors[i_runID], label=label, fill_alpha=0.1)

axs[0].set_xlim(binRange)
axs[0].set_ylim(0,3e3)
for i_pix in range(4):
    # axs[i_pix].set_xlim(binRange)
    title = PixNames[i_pix]
    axs[i_pix].set_title(PixNames[i_pix])

subtitles = ""
Hist.finalize(runs[runID], fig, axs, "ToT [ns]", "Entries", "ToT (krum_bias_trimming scan)", subtitles, measurement="Fe55_all", logy=False)
fig.savefig("output/ChargeCalibration/ToT_pixWise.pdf")
fig.savefig("output/ChargeCalibration/png/toT_pixWise.png", dpi=400)

In [ ]:
runIDs = arr([637,635,639,633,638,636,640,634])

rangeX = [5, 26]
rangeY = [0.05, 0.185]
binNX = 40
binNY = 30
NameX = "Risetime"
NameY = "Amplitude"
logScale = False

colX = M[NameX]
colY = M[NameY]
pixMap = [1,3,0,2]
PixNames = arr(["Pix (0,0)", "Pix (0,1)", "Pix (1,0)", "Pix (1,1)"])

risetime_regions = {
    637: [[6.,8],[6.,8],[6.,8],[6.,8]],
    635: [[6.,8.5],[6.,8.5],[6.,8.5],[6.,8.5]],
    639: [[5.5,7.5],[6.,8.5],[6.,8],[6.,8.5]],
    633: [[5.5,7.5],[6.,8],[5.5,7.5],[6.,8]],
    638: [[5.5,7.5],[5.5,8],[5.5,7.5],[5.5,8]],
    636: [[5.5,7],[5.5,8],[5.5,7.5],[5.5,8]],
    640: [[5.5,7],[5.5,8],[5.5,7.5],[5.5,8]],
    634: [[5.5,7],[5.5,7.7],[5.5,7.3],[5.5,7.7]],
}

for runID in runIDs:
    fig, axs = plt.subplots(4,4, sharex=False, sharey=False, figsize=[8,6], width_ratios=[1,.2,1,.2], height_ratios=[.2,1,.2,1])

    for i_row in range(2):
        for i_col in range(2):
            i_pix = i_row*2+i_col
            i_pix = pixMap[i_pix]
            mask = [True if x > 0 else False for x in runs[runID]["data"][:,i_pix,colX]]
            entriesX = runs[runID]["data"][mask,i_pix,colX]
            entriesY = runs[runID]["data"][mask,i_pix,colY]
            hist, binsx, binsy = np.histogram2d(entriesX, entriesY, bins=[binNX, binNY], range=[rangeX, rangeY])

            extent = [binsx[0], binsx[-1], binsy[0], binsy[-1]]

            axs[2*i_row+1, 2*i_col].imshow(hist.T, extent=extent, aspect="auto", origin='lower', cmap='viridis')
   
            axs[2*i_row+1, 2*i_col].axvline(risetime_regions[runID][i_pix][0], color="red", lw=.5, ls="--")
            axs[2*i_row+1, 2*i_col].axvline(risetime_regions[runID][i_pix][1], color="red", lw=.5, ls="--")
        
            # y-projection
            hist1d, bins = np.histogram(entriesY, bins=binNY, range=rangeY)
            axs[2*i_row+1, 2*i_col+1].set_xlim(0, 1.3*np.max(hist1d))
            axs[2*i_row+1, 2*i_col+1].set_ylim(rangeY)
            axs[2*i_row+1, 2*i_col+1].stairs(hist1d, bins, color="black", orientation="horizontal", alpha=1, fill=False)
            axs[2*i_row+1, 2*i_col+1].stairs(hist1d, bins, color="black", orientation="horizontal",alpha=0.2, fill=True)
    
            # x-projection
            hist1d, bins = np.histogram(entriesX, bins=binNX, range=rangeX)
            axs[2*i_row, 2*i_col].set_xlim(rangeX)
            axs[2*i_row, 2*i_col].set_ylim(0, 1.3*np.max(hist1d))
            axs[2*i_row, 2*i_col].stairs(hist1d, bins, color="black", alpha=1, fill=False)
            axs[2*i_row, 2*i_col].stairs(hist1d, bins, color="black", alpha=0.2, fill=True)

            # axs[2*i_row+1, 2*i_col].set_title(PixNames[i_pix], loc="right")
            title_str = PixNames[i_pix]+"\nN={:.1f}k".format(np.sum(hist)/1000)
            axs[2*i_row+1, 2*i_col].text(0.95, 0.95, title_str, transform=axs[2*i_row+1, 2*i_col].transAxes, ha="right", fontdict={"color":"white"}, va="top")

    Hist.finalize(runs[runID], fig, axs.flatten(), title=NameY+" vs "+NameX, xlabel=None, ylabel=None, subplots_adjust=[0.,0.,0.93])
    for ax in axs.flatten():
        ax.grid(False)
        ax.tick_params("both", direction="in", top=True, right=True)
    for i_row in range(3):
        for i_col in range(4):
            axs[i_row, i_col].set_xticklabels([])
            axs[i_col, i_row+1].set_yticklabels([])
    for i_row in range(2):
        for i_col in range(2):
            axs[2*i_row, 2*i_col].axis("off")
            axs[2*i_row, 2*i_col+1].axis("off")
            axs[2*i_row+1, 2*i_col+1].axis("off")
    for i in range(2):
        axs[2*i+1,0].set_ylabel("Amplitude [V]", loc="top")
        axs[-1, 2*i].set_xlabel("Risetime [ns]", loc="right")
    
    if False:
        fig.savefig("output/ChargeCalibration/Amplitude_vs_Risetime_"+str(runID)+".pdf")